In [1]:
import pandas as pd

class ShiftReduceParser:
    def __init__(self, grammar_rules, start_symbol):
        self.grammar = self.parse_grammar(grammar_rules)
        self.start_symbol = start_symbol
        self.stack = ['$']
        self.buffer = []

    def parse_grammar(self, grammar_rules):
        grammar = {}
        for rule in grammar_rules:
            if '->' not in rule:
                raise ValueError(f"Invalid grammar rule: {rule}")
            lhs, rhs = rule.split('->')
            lhs = lhs.strip()
            rhs = rhs.strip()
            if not lhs or not rhs:
                raise ValueError(f"Invalid grammar rule: {rule}")
            if lhs in grammar:
                grammar[lhs].append(rhs)
            else:
                grammar[lhs] = [rhs]
        return grammar

    def get_action(self):
        # Check for reduction
        for lhs, rhs_list in self.grammar.items():
            for rhs in rhs_list:
                rhs_tokens = rhs.split()
                if rhs_tokens == self.stack[-len(rhs_tokens):]:
                    return f"Reduce {lhs}->{rhs}"
        
        # Check for shift
        if self.buffer and self.buffer[0] != '$':
            return f"Shift {self.buffer[0]}"
        
        # Check for accept
        if len(self.stack) == 2 and self.stack[1] == self.start_symbol and self.buffer == ['$']:
            return "Accept"
        
        return None

    def shift(self):
        if not self.buffer:
            raise ValueError("Buffer is empty, cannot shift.")
        self.stack.append(self.buffer.pop(0))

    def reduce(self, production):
        lhs, rhs = production.split('->')
        rhs_tokens = rhs.split()
        if self.stack[-len(rhs_tokens):] != rhs_tokens:
            raise ValueError(f"Invalid reduction: {production}")
        self.stack = self.stack[:-len(rhs_tokens)] + [lhs]

    def parse_input(self, input_string):
        self.stack = ['$']
        self.buffer = input_string.split() + ['$']
        results = []

        while True:
            results.append({
                'Stack': ' '.join(self.stack),
                'Input Buffer': ' '.join(self.buffer),
                'Parsing Action': ''
            })

            action = self.get_action()

            if action is None:
                results[-1]['Parsing Action'] = "Error: No valid action found"
                break

            results[-1]['Parsing Action'] = action

            if action == "Accept":
                break
            elif action.startswith("Shift"):
                self.shift()
            elif action.startswith("Reduce"):
                try:
                    self.reduce(action.split(' ', 1)[1])
                except ValueError as e:
                    results[-1]['Parsing Action'] = f"Error: {str(e)}"
                    break
            else:
                results[-1]['Parsing Action'] = f"Error: Invalid action {action}"
                break

        return pd.DataFrame(results)

C:\Users\LENOVO\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\LENOVO\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
grammar_rules = [
    "S -> S + S",
    "S -> S * S",
    "S -> id"
]
start_symbol = "S"
input_string = "id + id + id"

parser = ShiftReduceParser(grammar_rules, start_symbol)
result = parser.parse_input(input_string)
display(result)

,Stack,Input Buffer,Parsing Action
0,$,id + id + id $,Shift id
1,$ id,+ id + id $,Reduce S->id
2,$ S,+ id + id $,Shift +
3,$ S +,id + id $,Shift id
4,$ S + id,+ id $,Reduce S->id
5,$ S + S,+ id $,Reduce S->S + S
6,$ S,+ id $,Shift +
7,$ S +,id $,Shift id
8,$ S + id,$,Reduce S->id
9,$ S + S,$,Reduce S->S + S


In [3]:
grammar_rules = [
    "E -> 2 E 2",
    "E -> 3 E 3",
    "E -> 4"
]

start_symbol = "E"
input_string = "3 2 4 2 3"

parser = ShiftReduceParser(grammar_rules, start_symbol)
result = parser.parse_input(input_string)
display(result)

,Stack,Input Buffer,Parsing Action
0,$,3 2 4 2 3 $,Shift 3
1,$ 3,2 4 2 3 $,Shift 2
2,$ 3 2,4 2 3 $,Shift 4
3,$ 3 2 4,2 3 $,Reduce E->4
4,$ 3 2 E,2 3 $,Shift 2
5,$ 3 2 E 2,3 $,Reduce E->2 E 2
6,$ 3 E,3 $,Shift 3
7,$ 3 E 3,$,Reduce E->3 E 3
8,$ E,$,Accept
